Notebook created by [Saku Peltonen](https://disco.ethz.ch/members/speltonen)

In [1]:
# Uncomment for Colab
# !pip install gymnasium pygame imageio swig
# !pip install "gymnasium[box2d]"

# Reinforcement learning

In this session, we will take a look at reinforcement learning (RL). The first part of the notebook introduces the basics of RL using traditional, table-based methods. In the second part, we apply deep neural networks as function approximators to solve RL tasks. 

Start by installing the required packages and downloading figures:

In [2]:
import ipywidgets as widgets
from IPython.display import display, Image
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np
import gymnasium as gym
import pygame
import os
import torch
import torch.nn as nn
import random
import imageio
import math
import shutil
import csv

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Q-learning in FrozenLake

<img src="https://polybox.ethz.ch/index.php/s/RGYpM0ETqPyUD0d/download" width="400" height="400" />

FrozenLake is a toy environment for training and testing RL agents. The goal of the player is to get to the target square (marked by the gift box) without falling into the holes in the ice. The player moves orthogonally. However, the ice is slippery, so the player does not always go in the intended direction! 

Let's try out the environment. The next cell contains utility functions. You do not need to go through it.

In [3]:
action_to_arrow = {0: '←', 1: '↓', 2: '→', 3: '↑'}

def get_board_dims(frozenlake_env):
    height, width = frozenlake_env.unwrapped.desc.shape
    observation_size = height * width
    return observation_size, height, width


class NoWindowWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        # Set up pygame with a dummy video driver to prevent window creation
        os.environ['SDL_VIDEODRIVER'] = 'dummy'
        pygame.init()
    def observation(self, *args):
        return self.env.observation(*args)

def interactive_play(env, evaluation=None, display_arrows=True, cell_size=1, use_keys=False):
    height, width = env.unwrapped.desc.shape
    plot_width = width * cell_size
    plot_height = height * cell_size

    def on_action(action):
        nonlocal move_count
        move_count += 1
        state, reward, terminated, truncated, info = env.step(action)

        update_view(state, reward=reward, terminated=terminated)

        return terminated

    def draw_arrows(ax, evaluation):
        for y in range(height):
            for x in range(width):
                state = y * width + x
                try: 
                    # In wrapped environments, we need the modified observation
                    observation = env.observation(state).to(device)
                except AttributeError:
                    observation = state
                    
                q_values = [evaluation(observation, action) for action in range(4)]

                text_rows = [f"{action_to_arrow[action]}: {str(round(q_values[action], 2))}" for action in range(4)]
                text = "\n".join(text_rows)
                plt.text(x / width + 1/(2*width), 
                         1 - (y / height + 1/(2*height)),  # NOTE: y-coordinates are flipped
                         text, fontsize=10, 
                         ha='center', va='center', 
                         color='black', 
                         alpha=1,
                         transform = ax.transAxes)



    def update_view(state, reward=0, terminated=False):
        clear_output(wait=True)
        
        fig, ax = plt.subplots(figsize=(plot_width, plot_height))
        ax.set_xticks([])
        ax.set_yticks([])

        ax.imshow(env.render())
        ax.set_xticks([])
        ax.set_yticks([])

        if display_arrows and evaluation is not None:
            draw_arrows(ax, evaluation)
            
        # plt.savefig('frozenlake_frame.png')
        plt.show()

        print("Previous reward: " + str(reward))
        if not terminated:
            print("Move count: ", move_count)
            if not use_keys:
                display(grid)
        else: 
            if reward == 1:
                print("Game won!")
            else: 
                print("Game lost!")


    env = NoWindowWrapper(env)
    state, _ = env.reset()
    move_count = 0
    

    if not use_keys:
        buttons = {}

        for action in range(4):
            button = widgets.Button(description=action_to_arrow[action])
            button.action_key = action
            # Attach the click event to the function
            button.on_click(lambda b: on_action(b.action_key))

            buttons[action] = button

        # Display the buttons in a grid layout
        grid = widgets.GridBox(
            children=[buttons[action] for action in range(4)],
            layout=widgets.Layout(
                width='700px',
                grid_template_columns='25% 25% 25% 25%',
                grid_template_rows='auto',
                grid_gap='5px'
            )
        )
    
    update_view(state)

    if use_keys:
        key_to_action = {'a': 0, 's': 1, 'd': 2, 'w': 3}

        while True:
            key = input("Move (W/A/S/D or Q to quit): ").lower()
            if key == 'q':
                break
            if key in key_to_action:
                terminated = on_action(key_to_action[key])
                if terminated:
                    break
    


def generate_gif(env, agent, n_frames=100, filename='animation', display_result=True):
    """Generates a gif of the agent playing. NOTE: this function is not targeted to be used for FrozenLake"""
    frames = []
    image_folder = "frames_temp"
    os.makedirs(image_folder, exist_ok=True)

    env = NoWindowWrapper(env)
    
    observation = env.reset()[0]

    total_reward = 0
    # Render the environment and save images
    for i in range(n_frames):  
        action = agent.act(observation, epsilon=0)
        next_observation, reward, done, _, _ = env.step(action)
        
        total_reward += reward
        frame = env.render()
        observation = next_observation

        frames.append(frame)
        
        # Save the frame as an image
        image_path = f"{image_folder}/frame_{i:04d}.png"
        imageio.imwrite(image_path, frame)
        
        if done:
            print(f"Total reward: {total_reward}")
            observation = env.reset()[0]
            total_reward = 0

    # Create a GIF from the saved frames
    image_files = [f"{image_folder}/frame_{i:04d}.png" for i in range(len(frames))]
    images = [imageio.imread(image_file) for image_file in image_files]
    imageio.mimsave(filename + '.gif', images, duration=0.05)

    # Remove the frames folder after creating the GIF
    shutil.rmtree(image_folder)

    if display_result:
        display(Image(filename='animation.gif'))

## Testing the environment

An instance of the environment is specified by the contents of the grid cells:
- start (S)
- frozen (F)
- holes (H)
- goal (G)

Programmatically we can represent it as a list of strings: 

In [4]:
desc = [
    'SFFF',
    'FHFH',
    'FFFH',
    'HFFG'
]


Let's first try the environment in a safe setting, where the slipperiness of the lake is turned off. 

Run the below cell to launch an instance of the environment. You can control the agent with the arrow buttons below.

In [5]:
env = gym.make('FrozenLake-v1', is_slippery=False, desc=desc, render_mode='rgb_array')
interactive_play(env)

## Use this version on Snowflake
# interactive_play(env, use_keys=True)

To restart the game, simply rerun the cell.

### Slippery lake

On a slippery lake, the player does not always move as intended, but may slip and move to the sides instead. For example, when trying to move up, the agent only moves up with a probability of 33%, while there is a 33% chance of moving right, and a 33% chance of moving left.  

The logic is easiest to understand by trying it out yourself: 

In [6]:
desc = [
    "SFFF",
    "FFFH",
    "FFFF",
    "FFFF",
    "HFFG"
]
env = gym.make('FrozenLake-v1', desc=desc, is_slippery=True, render_mode='rgb_array')

interactive_play(env)

## Use this version on Snowflake
# interactive_play(env, use_keys=True)

## Markov Decision Process

Before training an agent for the FrozenLake environment, we need to go through a bit of background. The formal model for reinforcement learning is the *Markov Decision Process* (MDP). A Markov decision process is defined by 
- $S$: a set of *states*. There is a specific *initial state* $s_{\text{initial}} \in S$ and a set of *final states* $S_{\text{final}} \subseteq S$. 
- $A$: a set of *actions*
- $T$: *transition function*. $T(s', a, s) = P(s' \mid s, a)$ gives the conditional probability of moving to state $s'$ when taking action $a$ in state $s$. In a *Markov process*, the transition depends only on the current state and action.
- $R$: *reward function*. $R(s', a, s)$ gives the reward for moving to state $s'$ by taking action $a$ in state $s$.

In *partially observable* environments, the agent does not know the full state of the environment. Instead, it receives an *observation* of the environment. For example, in Minesweeper a player only knows the numbers revealed by their moves, not where the mines are initially placed. 

### Example: FrozenLake
- $S = \{(x,y): x \in [\text{width}], y \in [\text{height}]\}$. This is a set of coordinates, specifying the position of the agent. 
- $A = \{$ left, up, right, down $\}$
- $R$: the reward is 1 for moving to the target square. By default, falling into a hole does not give a negative reward (but we could easily change that). 

The pictures below illustrate how the transition probabilities work: 

<img src="https://polybox.ethz.ch/index.php/s/f08unT640Ux6kDB/download" width="480" height="400" style="vertical-align: top; margin-right: 20px;"/>
<img src="https://polybox.ethz.ch/index.php/s/3coIl6Tvw0tQsRS/download" width="490" height="440" style="vertical-align: top;"/>

As you can see, if a move would take the player outside of the grid, the player remains in the current square. 

### Policy

We call a player acting in the environment an *agent*. An agent is specified by its *policy* $\pi: S \rightarrow A$, a map from states to actions. In a deterministic policy, the agent always picks the same action for a given state. Policies can also be stochastic (for example in the case of $\epsilon$-greedy exploration).

### Objective 

The goal in reinforcement learning is to find a policy that maximizes the *cumulative expected reward*: 
$$\mathbb{E} \left[\sum_{t=0}^\infty \gamma^t R(s_{t+1}, a_t, s_t) \right]$$
Here $s_t, a_t$ denote the state and action at time step $t$, respectively. The expectation is taken over the transitions: the next state $s_{t+1}$ is sampled according to the distribution $P(s_{t+1} | s_t, a_t)$.

$\gamma \in [0,1]$ is a *discount factor*. A low value of $\gamma$ encourages the agent to pay more attention to rewards in the near future. Conversely, when $\gamma$ is close to 1, distant rewards are almost as valuable. The role of $\gamma$ is important - the optimal policy usually depends on its value. 


### Test your skills

Once you feel sure on your feet, you can try a more difficult instance of the FrozenLake environment:

In [7]:
desc = [
    "FHHH",
    "SFFG",
    "FHHF",
    "FFFF"
]

env = gym.make('FrozenLake-v1', desc=desc, is_slippery=True, render_mode='rgb_array')
interactive_play(env)

## Use this version on Snowflake
# interactive_play(env, use_keys=True)

### Task 1:
<img src="https://polybox.ethz.ch/index.php/s/1Yb505EmKgEh1v0/download" width="400" height="400" />

In this exercise, we test your understanding of the environment, policy, and the effect of $\gamma$. In the environment above, what is the optimal policy when rewards are not discounted, i.e. $\gamma=1$? Be prepared to give a high level description of the policy. An intuitive explanation for why the policy is optimal is enough (there is no need to actually calculate the expected reward).

*Hint:* moving towards the wall is allowed. This is sometimes beneficial to avoid falling into a hole. 

**Optional challenge question:** What is the optimal policy when $\gamma$ is very small, e.g. $\gamma=0.1$?

When gamma = 1, we simply care that we reach the goal and not how many steps we take. When gamma is very small, we search for the most optimal/fastest way to reach the goal.

## Introduction to the gymnasium library

In this notebook, we use the [gymnasium library](https://gymnasium.farama.org/index.html) (gym for short), originally developed by OpenAI for training and testing RL agents. FrozenLake is one of many prebuilt environments in the library. The core benefit of gym is the standardized environment interface. This means that we can easily write agents that can be trained in different environments, without requiring major modifications to the agent. 



Every gym environment has these four methods implemented: 
- `step`: Takes a given action in the environment. Returns an observation, reward, flag for reaching a final state, and some other information. 
- `reset`: Puts the environment in its initial state. Returns an observation. 
- `render`: Displays the state of the environment. 
- `close`: Closes the environment. This is only needed when we render the environment using external libraries. 

Please see the [documentation](https://gymnasium.farama.org/api/env/) for more information. 

### Spaces

Spaces are used to describe actions and observations. The simplest type of space is given by the `Discrete` class, which describes a single number, usually from 0 to $n-1$. Later in this notebook we will have environments with non-discrete observations. There we use the `Box` space, which is a box in $\mathbb{R}^n$ (cartesian product of closed intervals). In FrozenLake, the action space consists of the 4 possible actions (left, up, right, down) encoded as integers:

In [8]:
env.action_space

The observation space is encoded as integers $0, \dots, \text{width} \times \text{height}$:

In [9]:
env.observation_space

*Note:* we can use the term state and observation interchangeably in FrozenLake, because the environment is *fully observable*, that is, observations are equal to the state of the environment. 

## Q-learning

*Q-learning* is an RL algorithm that tries to learn the value of taking an action in a particular state. This value, denoted $Q(s,a)$, includes the reward from the next step, as well as the discounted reward from all future steps. 

In practice, we can think of $Q$ as a two-dimensional list `Q` of numbers with size $|S| \times |A|$, where `Q[s,a]` contains the respective state-action value:

In [10]:
Q = np.random.random((env.observation_space.n, env.action_space.n))  # filled with random values for illustration purposes
Q

Why is knowing $Q(s,a)$ useful? If we knew the correct state-action values, we could set our policy $\pi$ to play the action $a = \text{argmax}_{a \in A} Q(s,a)$, i.e.  the action that maximizes the expected reward. If the state-action values are correct, this policy is **the best we can do**.

### Task 2: Greedy policy
In this task, you need to implement a simple policy that picks the action with the best Q-value. Given a state $s$ and the Q-table, return the action $\text{argmax}_{a \in A} Q(s,a)$. In case multiple actions have the same Q value, the function can return one of them arbitrarily.

In [11]:
##### Exercise #####

def act_greedy(state, Q):
    action = Q[state].argmax()
    return action
print(act_greedy(0, Q))

### Q-learning algorithm
Obviously $Q(s,a)$ is not actually known, and computing it explicitly is veeery expensive because we would need to consider all possible state-action sequences. 

The Q-learning algorithm tries to learn the Q-values. Suppose we have some initial estimate of the $Q$ function (which could even be a zero-initialized array). Suppose we take an action $a$ in state $s$, get some reward $r$, and end up in state $s'$. Observing a reward and the next state gives a new estimate for $Q(s,a)$:

$$r + \gamma \max_{a' \in A} Q(s', a')$$

where $\gamma$ is the discount factor for future rewards, and $\max_{a' \in A} Q(s', a')$ is the estimated future rewards from state $s'$ onwards, when we take the action $a'$ that seems to give the best rewards. Using this, we can update our original estimate: 

$$Q^{new}(s,a) \leftarrow (1-\alpha) Q(s,a) + \alpha \cdot \left(r + \gamma \max_{a' \in A} Q(s', a')\right)$$
Here $\alpha \in [0,1]$ is a learning rate parameter that controls how gradually we update the estimate. A higher learning rate gives more weight to the new estimate $r + \gamma \max_{a' \in A} Q(s', a')$. 


### Task 3:
Implement the Q-learning update as described above. 

*Note:* If `terminated=True`, the value of $r + \gamma \max_{a' \in A} Q(s', a')$ should only be $r$

In [12]:
##### Exercise #####

def learn_from_experience(Q, state, action, reward, next_state, terminated, alpha, gamma):
    if terminated == False:
        Q[state][action] = (1-alpha) * Q[state][action] + alpha * (reward + gamma * max(Q[next_state]))
    else:
        Q[state][action] = (1-alpha) * Q[state][action] + reward

### Exploration vs exploitation

In this section, we look at the tradeoff between *exploring* new strategies and *exploiting* our current knowledge. 

Recall that selecting $a = \text{argmax}_{a \in A} Q(s,a)$ is the best possible policy **if** we know the $Q$-function exactly. However, we are only in the process of learning $Q$. 
Using the greedy policy by selecting $a = \text{argmax}_{a \in A} Q(s,a)$ means taking advantage of what we currently know. However, the $Q$-values may be inaccurate, so there may be undiscovered value in taking some other action $a'$. In other words, using the greedy policy may get our agent stuck in local optima. 

#### $\epsilon$-greedy exploration

To balance exploitation, we need to do *exploration* to discover new strategies. The simplest way to do this is to simply try random moves. An $\epsilon$-greedy exploration policy does this by 
- selecting a random action $a \in A$ with probability $\epsilon$
- using $a = \text{argmax}_{a \in A} Q(s,a)$ with probability $1-\epsilon$ 

### Task 4
Implement the $\epsilon$-greedy exploration policy

In [13]:
##### Exercise #####

def epsilon_greedy_policy(state, Q, epsilon):
    actions = np.arange(Q.shape[1])  # [0, 1, 2, 3]
    if random.random() < epsilon:
        action = np.random.choice(actions)
    else:
        action = act_greedy(state, Q)
    return action
epsilon_greedy_policy(0, Q, 0.5)

### $\epsilon$-scheduler

The value of the $\epsilon$ is controlled through the training process. Usually, we start with a higher value to give more priority to exploration, and gradually decrease $\epsilon$ as training progresses. We use a linear scheduler:

In [14]:
def get_epsilon(episode, n_episodes, epsilon_start=1, epsilon_end=0):
    return epsilon_end + (epsilon_start - epsilon_end) * (1 - episode / n_episodes)

## Training an agent

We will combine all the tools built above to train an agent. We will use the following FrozenLake instance. 

To make sure that the agent doesn't get stuck, we limit the maximum number of actions taken by `max_steps`. 

In [15]:
desc = [
    "SFFH",
    "FHFF",
    "FFFF",
    "FFFH",
    "HFFG"
]

# Please don't change these values
max_steps = 300
gamma = 0.99

env = gym.make('FrozenLake-v1', desc=desc, is_slippery=True, render_mode='rgb_array', max_episode_steps=max_steps)

In [16]:
# interactive_play(env)

## Use this version on Snowflake
# interactive_play(env, use_keys=True)

You can uncomment the above cell to try the map. There exists a policy that always succeeds, and our agent should be able to find it (can you?). 

### Run a single episode
The below function should run a single game (episode) in the environment. 

In [17]:
def run_episode(env, Q, gamma, epsilon, alpha):
    """
    Run a single episode of the game using the Q-learning algorithm.

    Args:
        env: gym environment
        Q: numpy array of shape (n_states, n_actions)
        gamma: float
        max_steps: int
        epsilon: float
        alpha: learning rate (float)
    
    Returns:
        total_reward: float. For FrozenLake this is 1 for reaching the goal, 
            0 for falling into a hole or running out of steps)
        lost: bool, indicating whether the agent fell into a hole
        truncated: bool, indicating whether the episode was ended due to reaching max_steps
    """

    state, info = env.reset()
    total_reward = 0
    done = False

    while not done:
        action = epsilon_greedy_policy(state, Q, epsilon)
        new_state, reward, terminated, truncated, info = env.step(action)
        
        learn_from_experience(Q, state, action, reward, new_state, terminated, alpha, gamma)
        
        total_reward += reward
        state = new_state

        # End the episode if the agent reaches the goal or falls into a hole (terminated=True)
        # or if the episode reaches the maximum number of steps (truncated=True)
        done = terminated or truncated

    lost = total_reward == 0 and (not truncated)  # NOTE: this only holds for FrozenLake
    return total_reward, lost, truncated

### Main loop

The main loop simply runs the above function `n_episodes` times to train the agent. Additionally, every `test_interval` episodes, we run a few test episodes with $\epsilon=0$ to see how well the current greedy policy performs. 

In [18]:
def train_Q_agent(env,
                  Q,
                  n_episodes,  
                  test_interval, 
                  n_test_episodes, 
                  lr, 
                  gamma, 
                  epsilon_start=1, 
                  epsilon_end=0):

    episode = 0
    while episode < n_episodes + 1:
        if episode % test_interval == 0:
            results = [run_episode(env, Q, gamma, 0, lr) for _ in range(n_test_episodes)]
            test_rewards = [result[0] for result in results]
            lost = [result[1] for result in results]
            truncated = [result[2] for result in results]
            print(
                f"Episode: {episode}, won: {round(100*np.mean(test_rewards),2)}%",
                f"lost: {round(100* sum(lost) / n_test_episodes,2)}%",
                f"truncated: {round(100* sum(truncated) / n_test_episodes,2)}%"
            )

        else:
            epsilon = get_epsilon(episode, n_episodes, epsilon_start, epsilon_end)
            _ = run_episode(env, Q, gamma, epsilon, lr)
        
        episode += 1

### Task 5: Training the agent

Below we define hyperparameters and call the main training loop. The default hyperparameters should work, assuming your solutions to the previous tasks are correct. 

**The agent should find a strategy that works at least 99% of the time**. The optimal policy never falls into holes on this map, but there is a very small chance that it does not reach the goal within `max_steps` steps.

In [19]:
##### Exercise #####

n_episodes = 30000
alpha = 0.01  # learning rate
epsilon_start = 1
epsilon_end = 0
test_interval = n_episodes // 20
n_test_episodes = 30

observation_size, _, _ = get_board_dims(env)
n_actions = 4

# Initialize the Q table
Q = np.zeros((observation_size, n_actions))

# Train the agent
train_Q_agent(env, Q, n_episodes, test_interval, n_test_episodes, 
              alpha, gamma, epsilon_start=epsilon_start, epsilon_end=epsilon_end)

### Inspect the Q-values
After training an agent, you can open the interactive environment and inspect the learned Q-values. 

*Note:* You can simulate your agent by selecting the action with the highest value.

In [20]:
# Helper function to access state-action values 
evaluation = lambda state, action: Q[state, action]

interactive_play(env, evaluation=evaluation, display_arrows=True)

## Use this version on Snowflake
# interactive_play(env, evaluation=evaluation, display_arrows=True, use_keys=True)

### Debugging tips (optional)

Debugging RL agents can be a bit tricky. Here's a few suggestions to try in case your agent is not doing well:

- **Inspect the Q-values (see above):** The values should all be within $[0,1]$, because the largest possible total reward is 1 and there are no negative rewards. 
- **Training the agent in a simpler environment:** You can try to turn off the slipperiness by setting `slippery=False` or use a simpler map. For this, you may also try changing the value of $\gamma$ because the Q-values can be easier to interpret when $\gamma$ is small. Remember to set $\gamma=0.99$ once you're done with the tests. Here's a few environments you can try:

In [21]:
desc = [
    "SFFG"
]

desc = [
    "SFFF",
    "FFFF",
    "FFFF",
    "FFFG"
]

- **Printing:** You can also try to print the loss (difference between $Q(s,a)$ and the updated estimate), the value of $\epsilon$, and the number of steps taken in each episode. 

# Deep Reinforcement Learning

Table-based Q-learning works well if the number of observations (states) and actions is limited. This restricts table-based Q-learning to environments where observations are discrete. While there are some workarounds for this (discretization), there is a more fundamental issue in table-based methods:

### Curse of dimensionality
In complex environments, observations usually contain information in multiple dimensions. For example, consider the [Humanoid environment](https://gymnasium.farama.org/environments/mujoco/humanoid/), where a bipedal robot learns to walk. In this environment, each observation includes coordinates, velocities and angles between various body parts, for a total of 376 dimensions! Even if each dimension is represented by a few discrete values (e.g. 10), we would need a $Q$-table with $10^{376}$ rows!

<img src="https://gymnasium.farama.org/_images/humanoid.gif" width="400" height="400" />

## Deep Q-Networks

The idea in deep Q-learning is to replace the Q-table with a deep Q-network (DQN), which is simply a deep neural network for approximating the Q-values. We give the observation as input to the network, and it returns Q-values for each action.

## Experience Replay

We modify the training loop by storing transitions in the environment as  *experiences* in a replay buffer. An experience is a tuple $e = (\text{observation}_t, \text{action}_t, \text{reward}_t, \text{observation}_{t+1}, \text{terminated}_{t+1})$, describing a single step in the environment. To train the model, we randomly sample a minibatch from the replay buffer and run backpropagation on the minibatch. There are two major benefits in using a replay buffer: 
1. **Stability**: Experiences randomly sampled from a large buffer represent a broad sample, whereas consequtive experiences from the environment are heavily correlated.
2. **Cost**: Sometimes generating experiences from the environment is more expensive than training the model with those experiences. Using a replay buffer allows us to reuse experiences. 

In [22]:
from collections import deque, namedtuple

Experience = namedtuple('Experience',
                        ('observation', 'action', 'reward', 'next_observation', 'terminated'))
# terminated = won or lost. truncation does not affect this
# handling this detail correctly is quite important

class ExperienceReplay:
    def __init__(self, capacity, batch_size):
        self.buffer = deque(maxlen=capacity)
        self.batch_size = batch_size

    def push(self, *args):
        """Push new experience to the memory.
        As the buffer is a deque with a fixed maxlen, oldest experiences will be automatically discarded"""
        self.buffer.append(Experience(*args))

    def sample(self):
        """Sample a batch of experiences from the memory."""
        assert len(self.buffer) >= self.batch_size
        batch = random.sample(self.buffer, self.batch_size)
        observations, actions, rewards, next_observations, terminateds = zip(*batch)

        observations = torch.stack(observations).to(device)  # (batch_size, observation_size)
        actions = torch.LongTensor(actions).to(device)  # (batch_size,)
        rewards = torch.FloatTensor(rewards).to(device)  # (batch_size,)
        next_observations = torch.stack(next_observations).to(device)  # (batch_size, observation_size)
        terminateds = torch.BoolTensor(terminateds).to(device)  # (batch_size,)

        return observations, actions, rewards, next_observations, terminateds

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.buffer)

The below function runs a single episode. It looks very similar to what we used before, the only difference being saving the experience to the replay buffer instead of training on it directly.

In [23]:
def run_episode(env, agent, epsilon, train=True, add_to_memory=True):
    """
    Run a single episode.

    Args:
        env: gym environment
        agent: DQNAgent
        epsilon: float
        train: bool, whether to train the agent
    
    Returns:
        total_reward: float
    """

    observation, info = env.reset()
    total_reward = 0
    done = False
    episode_length = 0
    losses = []

    while not done:
        action = agent.act(observation, epsilon)
        new_observation, reward, terminated, truncated, _ = env.step(action)
        
        if add_to_memory:
            agent.memory.push(observation, action, reward, new_observation, terminated)
            
        if train:
            loss = agent.optimize_model()
            losses.append(loss)

        total_reward += reward
        observation = new_observation
        episode_length +=1
        agent.step_count += 1

        done = terminated or truncated

    losses = [loss for loss in losses if loss is not None]
    average_loss = sum(losses) / max(len(losses),1)

    return total_reward, episode_length, average_loss

### DQN for FrozenLake

We will first apply the DQN agent to the FrozenLake environment. This is a good starting point because debugging the agent in this simple environment is relatively easy. 


### Wrappers

Currently observations (states) are given as an integer in $0, \dots, \text{width} \cdot \text{height}$. This is not the most convenient representation for deep neural networks, which usually work better with vector inputs, so we transform the observation to a one-hot encoded representation. 

We can do this using a [wrapper](https://gymnasium.farama.org/api/wrappers/). Wrappers are a general tool in the gym library to modify some aspect of the environment, without altering the code of the environment or the agent directly. In fact, we already used a TimeLimit wrapper before to impose a bound on the number of steps. 

In [24]:
class OneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(OneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Discrete)
        n = env.observation_space.n
        self.n = n

        self.observation_space = gym.spaces.Box(
            low=0, high=1, shape=(n,), dtype=np.float32
        )

    def observation(self, obs):
        one_hot_obs = np.zeros(self.n)
        one_hot_obs[obs] = 1.0
        return torch.tensor(one_hot_obs).float()

Let's see what this does in practice:

In [25]:
desc = [
    "SFFH",
    "HFFG"
]

env = gym.make('FrozenLake-v1', desc=desc, render_mode='rgb_array')
env = OneHotWrapper(env)

# env.reset returns the initial state / observation
obs = env.reset()[0]
obs = obs.to(device)
obs

### Network

In principle, we can use any deep neural network (with the appropriate input and output dimensions) as the Q-network. For simplicity, let's not use a deep neural network right away. Rather, we will apply a very shallow network, with no hidden layers at all:

In [26]:
# Compute number of squares on the board
observation_size, _, _ = get_board_dims(env)

# Linear layer with 4 outputs corresponding to the different actions
net = nn.Linear(observation_size, 4)
net = net.to(device)

Feeding the observation to the network gives the Q-values

In [27]:
net(obs)

Note that with a single linear layer and a one-hot encoded input, each  state-action pair corresponds to a weight in the network. 


### Training the Deep Q-network

Now that we use a neural network for the Q-values, we need to train it using backpropagation. Recall the Q-value update from before 

$$Q^{new}(s,a) \leftarrow (1-\alpha) Q(s,a) + \alpha \cdot \left(r + \gamma \max_{a' \in A} Q(s', a')\right)$$

We think of $Q(s,a)$ as the *current estimate*, and $r + \gamma \max_{a' \in A} Q(s', a')$ as the *target estimate*. The target is analogous to the *target* $y$ in supervised learning.

Training involves minimizing the difference between the current and the target estimate using a loss function. Typical choices include the L2 loss $(x-y)^2$ or the L1 loss $|x-y|$. We use [SmoothL1Loss](https://pytorch.org/docs/stable/generated/torch.nn.SmoothL1Loss.html), which is a combination of the L1 and L2 losses. 

After computing the loss, we run backpropagation to move the current estimate $Q(s,a)$ closer to the target. **Gradients are only applied to the  current estimate and not the target.** It may be helpful to think of this in terms of supervised learning, where we apply gradients to the *model prediction* (current estimate), and not the target $y$ (target estimate). In practice, the target should be computed inside a `torch.no_grad()` block. 

#### Implementing the loss function

Let's first generate some experiences to see what a minibatch will look like:

In [28]:
memory = ExperienceReplay(capacity=10000, batch_size=4)

# We use a random agent for illustration purposes
class RandomAgent:
    def __init__(self, env, memory):
        self.env = env
        self.memory = memory
        self.step_count = 0

    def act(self, observation, epsilon):
        return self.env.action_space.sample()

agent = RandomAgent(env, memory)

env.reset()
for _ in range(10):
    run_episode(env, agent, 0, train=False)

Now we can sample a minibatch of experiences. You can use the next cell to see the type of tensors the loss computation will take as input:

In [29]:
observations, actions, rewards, next_observations, terminateds = memory.sample()
observations

### Task 6: Calculate loss

Your task is to implement the loss computation. The comments provide a basic structure. You can assume `batch_size` $\ge 2$

In [30]:
##### Exercise #####

def _compute_DQN_loss(observations, actions, rewards, next_observations, terminateds, net, gamma, criterion):
    """
    Compute the loss for a batch of experiences.

    Args:
        observations: torch tensor of shape (batch_size, observation_size)
        actions: torch tensor of shape (batch_size,)
        rewards: torch tensor of shape (batch_size,)
        next_observations: torch tensor of shape (batch_size, observation_size)
        terminateds: torch tensor of shape (batch_size,)
        net: DQN
        gamma: float
        criterion: loss function

    Returns:
        loss: scalar tensor 
    """
    ## YOUR CODE HERE

    # 1. Compute the Q values for the current observations-action pairs
    #   - use the net to compute the Q values for the current observations 
    #   - select the Q values corresponding to the played actions
    # Hint: for handling torch.tensors, the gather, unsqueeze and squeeze methods may be useful
    
    Q = net(observations)
    Q = Q.gather(1, actions.unsqueeze(1)).squeeze(1)

    with torch.no_grad():
        # 2. Compute the target estimate (see the formula above)
        #   - compute Q values for the next observations (max over actions)
        #   - multiply by gamma
        #   - set the Q values for terminated states to 0
        #   - add rewards
        next_Q = net(next_observations)
        next_Q = next_Q.max(dim=1).values * gamma
        next_Q[terminateds] = 0
        next_Q += rewards
        

    # 3. Use `criterion` to compute the loss and return it
    loss = criterion(Q, next_Q)
    return loss

The below cell allows you to test the loss computation (mainly to see if there are shape mismatches or type errors).

In [31]:
_compute_DQN_loss(observations, actions, rewards, next_observations, terminateds, net, 0.9, nn.SmoothL1Loss())

### Agent class

The agent is implemented in a `DQNAgent` class below. The class has two main methods: 
- `act`: Takes in an observation and epsilon, and acts according to the $\epsilon$-greedy policy. This is similar to the $\epsilon$-greedy policy implemented for the table-based agent. Note that we use `torch.no_grad` to not track the gradient here, because we only need the forward pass when generating experiences. 
- `optimize_model`: This is called in every iteration of the training loop. The method samples a minibatch from the experience replay, computes loss for the minibatch (using `_compute_DQN_loss` implented above) and runs backpropagation to train the model. 

In [32]:
class DQNAgent:
    def __init__(self, net, n_actions, compute_loss, gamma=0.98, lr=0.01, memory_capacity=100000, batch_size=16):
        self.net = net
        self.n_actions = n_actions
        self.gamma = gamma
        self.lr = lr
        self.batch_size = batch_size
        self.memory = ExperienceReplay(capacity=memory_capacity, batch_size=batch_size)
        self.compute_loss = compute_loss
        self.optimizer = torch.optim.Adam(net.parameters(), lr=lr)
        self.criterion = nn.SmoothL1Loss()
        self.step_count = 0
        
    def act(self, observation, epsilon):
        """
        Select an action according to an epsilon-greedy policy.

        Args:
            observation: torch.tensor of shape (observation_size,)
            epsilon: float

        Returns:
            action: int
        """
        if np.random.rand() < epsilon:
            return np.random.choice(range(self.n_actions))
        else: 
            observation = observation.to(device)

            # NOTE: torch.no_grad() disables gradient tracking.
            # We only need the forward pass, so this saves memory and computation
            with torch.no_grad():
                return self.net(observation).argmax().item()
            

    def optimize_model(self):
        if len(self.memory) < self.batch_size:
            return
        
        observations, actions, rewards, next_observations, terminateds = self.memory.sample()
        
        # Compute the loss
        loss = self.compute_loss(observations, actions, rewards, 
                                next_observations, terminateds, 
                                self.net, self.gamma, self.criterion)

        # Optimize the model
        self.optimizer.zero_grad()
        loss.backward()
        # In-place gradient clipping
        torch.nn.utils.clip_grad_value_(self.net.parameters(), 100)
        self.optimizer.step()

### Training loop

In [33]:
def train_DQN_agent(env,
                    agent,
                    n_episodes,  
                    test_interval, 
                    n_test_episodes, 
                    epsilon_scheduler):
    episode = 0
    best_test_reward = -math.inf

    while episode < n_episodes:
        if episode % test_interval == 0:
            test_rewards = [run_episode(env, agent, 0, train=False, add_to_memory=False)[0] for _ in range(n_test_episodes)]
            average_test_reward = np.mean(test_rewards)
            print(f"Episode: {episode}, average test reward: {(average_test_reward)}")

            if average_test_reward > best_test_reward:
                print('New best test reward. Saving model')
                best_test_reward = average_test_reward
                torch.save(agent.net.state_dict(), 'policy_net.pth')

        else:
            epsilon = epsilon_scheduler(episode)
            total_reward, episode_length, loss = run_episode(env, agent, epsilon, episode)
        
        episode += 1

Please use the following environment. For debugging, you can also use simpler environments or turn off slipperiness.

In [34]:
desc = [
    "SFFF",
    "FHFF",
    "FFFF",
    "FFFH",
    "HFFG"
]
max_steps = 300
gamma = 0.99

env = gym.make('FrozenLake-v1', desc=desc, is_slippery=True, render_mode='rgb_array', max_episode_steps=max_steps)
env = OneHotWrapper(env)

These parameters should work, but you can also try changing them:

In [35]:
n_episodes = 2000
lr = 0.002
epsilon_start = 1
epsilon_end = 0
test_interval = n_episodes // 10
n_test_episodes = 30
memory_capacity = 10000

n_actions = 4
observation_size, _, _ = get_board_dims(env)

net = nn.Linear(observation_size, n_actions).to(device)
agent = DQNAgent(net, n_actions, _compute_DQN_loss, gamma=gamma, lr=lr, memory_capacity=memory_capacity)
# Use a linear epsilon schedule
epsilon_scheduler = lambda episode: epsilon_end + (epsilon_start - epsilon_end) * (1 - episode / n_episodes)

train_DQN_agent(env, agent, n_episodes, test_interval, n_test_episodes, epsilon_scheduler)

You can use the cell below to inspect the Q-values

In [36]:
def get_q_values(obs, action):
    """Helper function to access state-action values."""    
    return agent.net(obs)[action].item()

interactive_play(env, evaluation=get_q_values, display_arrows=True)

## Use this version on Snowflake
# interactive_play(env, evaluation=get_q_values, display_arrows=True, use_keys=True

## Beyond Basic DQN

In this section, we will build upon the basic DQN model by introducing more advanced techniques, such as implementing a simple Q-network and optimizing it with Double DQN.

### Task 7: Network architecture

In order to facilitate parameter tuning, we implement a custom MLP class for the Q-network. The `hidden_size` parameter is a variable-length list of integers, describing the number of units for each hidden layer. The `activation` parameter can be used to control the activation function. 

**Note**: the last layer should not have an activation; the range of Q-values shouldn't be limited, since they can also be negative for example.

In [37]:
##### Exercise #####

class QNet(nn.Module): 
    def __init__(self, observation_size, n_actions, hidden_size=[128,128], activation=nn.ReLU):
        """
        Initialize the network.

        Args:
            observation_size: int, dimension of the observation vector
            n_actions: int, number of actions
            hidden_size: [int], the number of units for each hidden layer
            activation: 
        """
        super(QNet, self).__init__()

        ## YOUR CODE HERE
        ## Define the layers of the network
        ##  - create a list of layers and activations 
        ##       - use activation() to instantiate the activation function
        ##  - use nn.Sequential 

        layers = []
        input_dim = observation_size

        for hidden_dim in hidden_size:
            layers.append(nn.Linear(input_dim, hidden_dim))
            layers.append(activation())
            input_dim = hidden_dim

        layers.append(nn.Linear(input_dim, n_actions))

        self.net = nn.Sequential(*layers)
        self.net.to(device)


    def forward(self, x):
        return self.net(x)

## Double DQN

Training DQN agents can be an unstable process. The instability is often linked to the overestimation of Q-values. To address this issue, we implement a technique known as [Double DQN](https://arxiv.org/abs/1509.06461). 

Double DQN uses two networks: a *policy network* $Q_{\text{policy}}$ and a *target network* $Q_{\text{target}}$. Both networks share the same architecture and are initialized with identical weights. The policy network is what we are actively training. The target net is a slightly delayed version of the policy net. It's weights are updated by copying the weights from the policy net every once in a while. 

The target net is used to stabilize the update equation (copied from above):

$$Q(s,a) \leftarrow (1-\alpha) Q(s,a) + \alpha \cdot \left(r + \gamma \max_{a' \in A} Q(s', a')\right)$$

In Double DQN, we first select the best action in the next state using the policy net: $a^* = \arg\max_{a' \in A} Q_{\text{policy}}(s',a')$. The value of this action is then evaluated with the target net. The modified equation looks like this: 

$$Q_{\text{policy}}(s,a) \leftarrow (1-\alpha) Q_{\text{policy}}(s,a) + \alpha \cdot \left(r + \gamma Q_{\text{target}}(s', a^*)\right)$$



Every once in a while, we sync the two networks by setting $Q_{\text{target}} \leftarrow Q_{\text{policy}}$. More generally, the networks can be gradually synchronized by setting $Q_{\text{target}} \leftarrow (1-\tau) Q_{\text{target}} + \tau Q_{\text{policy}}$ where $\tau \in ]0,1]$.

### Task 8: Double-DQN Loss


In [38]:
##### Exercise #####

def _compute_DoubleDQN_loss(observations, actions, rewards, next_observations, terminateds, net, target_net, gamma, criterion):
    """
    Compute the loss for a batch of experiences.

    Args:
        observations: torch tensor of shape (batch_size, observation_size)
        actions: torch tensor of shape (batch_size,)
        rewards: torch tensor of shape (batch_size,)
        next_observations: torch tensor of shape (batch_size, observation_size)
        terminateds: torch tensor of shape (batch_size,)
        net: nn.Module
        target_net: nn.Module
        gamma: float
        criterion: loss function

    Returns:
        loss: scalar tensor
    """

    # 1. Compute the Q values for the current observations-action pairs
    # (You can copy your solution from Task 6, Part 1 here)
    Q = net(observations)
    Q = Q.gather(1, actions.unsqueeze(1)).squeeze(1)

    with torch.no_grad():
        # 2. Compute the target estimate (see the formula above)
        #   - find actions maximizing Q-values in the next states using the policy net
        #   - compute Q-values for these actions using the target net 
        #   - multiply by gamma
        #   - set the Q values for terminated states to 0
        #   - add rewards
        policy_Q = net(next_observations)
        next_actions = policy_Q.argmax(dim=1)
        target_Q = target_net(next_observations)
        next_Q = target_Q.gather(1, next_actions.unsqueeze(1)).squeeze(1)
        next_Q = next_Q * gamma
        next_Q[terminateds] = 0
        next_Q += rewards
        

    # 3. Use `criterion` to compute the loss and return it
    # (You can copy your solution from Task 6, Part 3 here)
    loss = criterion(Q, next_Q)
    return loss

The below cell generates batches of random experiences for you to test your implementation: 

In [39]:
memory = ExperienceReplay(capacity=10000, batch_size=4)

# We use the random agent from before for illustration purposes
agent = RandomAgent(env, memory)

env.reset()
for _ in range(10):
    run_episode(env, agent, 0, train=False)

# sample a batch
observations, actions, rewards, next_observations, terminateds = memory.sample()

net = QNet(observation_size, n_actions)
target_net = QNet(observation_size, n_actions)

_compute_DoubleDQN_loss(observations, actions, rewards, next_observations, terminateds, net, target_net, 0.9, nn.SmoothL1Loss())

### Double-DQN agent class

The agent class for Double-DQN can be almost identical to `DQNAgent`. Additionally, the target network should be synced with the policy network every `update_tgt_interval` steps (e.g. in the `compute_loss` method). This can be achieved with the following code 

In [40]:
# for target_param, param in zip(self.target_net.parameters(), self.policy_net.parameters()):
#     target_param.data.copy_(self.tau * param.data + (1.0 - self.tau) * target_param.data)

In the challenge, it's strongly recommended to apply Double-DQN (if you're using a DQN-based agent). To conclude the notebook, let's shift focus to a fundamentally different approach to RL.

## Policy-Based Reinforcement Learning

So far, we've explored *value-based* approaches, where the agent learns a value function that estimates expected future rewards for each state-action pair. But does it always make sense to assign precise values to everything? 

### A self-driving example

Suppose we are training a self-driving car. The agent might learn action-values like:
- Q(drive straight) = **1**
- Q(swerve into a pedestrian) = **-100**

The values feel a bit arbitrary. Does the *exact value* of these actions really matter? 

**The core issue:** Value-based methods assume everything can be compared on a numerical scale. However, in many cases, we only care about *relative preferences* — e.g., "swerve into a pedestrian is worse than driving straight" — not the exact numbers.


### Understanding Policy-Based Methods

Policy-based methods take a different route. Instead of relying on a Q-function, they directly learn a policy – a probability distribution over actions. 

The policy $\pi_{\theta}$ is parameterized by $\theta$. The goal is to adjust $\theta$ to increase the expected return $J(\theta)$, which is done with the following:


$$\nabla_\theta J(\theta) = E_{\tau \sim \pi_{\theta}} \sum_{t=0}^T \nabla_{\theta} \log \pi_{\theta}(a_t \mid s_t) \cdot R_t$$

where
- the expectation is over *trajectories* $\tau$ – the sequence of states, actions and rewards in an episode, with actions sampled from $\pi_\theta$
- $\pi_\theta(a_t \mid s_t)$ is the probability of taking the chosen action $a_t$ at step $t$
- $R_t = \sum_{k=t}^T \gamma^{k-t} r_{k}$ is the total discounted reward starting from step $t$

We use this gradient to update the policy in the direction that increases expected rewards:
$$ \theta \leftarrow \theta + \alpha \nabla_\theta J(\theta)$$
where $\alpha$ is the learning rate.

This is the essence of the REINFORCE algorithm. Let's implement it:
 

In [84]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
import matplotlib.pyplot as plt
from IPython.display import clear_output
import pygame


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class NoWindowWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        # Set up pygame with a dummy video driver to prevent window creation
        os.environ['SDL_VIDEODRIVER'] = 'dummy'
        pygame.init()
    def observation(self, *args):
        return self.env.observation(*args)

#### LunarLander environment

Our agent is controlling a spacecraft landing on the moon. Check the [documentation](https://gymnasium.farama.org/environments/box2d/lunar_lander/).

<img src="https://gymnasium.farama.org/_images/lunar_lander.gif" width="400" height="400" />

In [85]:
# Create the LunarLander environment
env = gym.make("LunarLander-v3", render_mode="rgb_array")
env = NoWindowWrapper(env)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

In [86]:
state_dim, action_dim
assert(state_dim == 8)

### Task 9: Policy Network

We need a policy network mapping. The network takes in an observation with `state_dim` dimensions and outputs probabilities for each of the `action_dim` actions. An MLP with a few layers should be enough. The last layer of the network should have a softmax activation. 

In [87]:
##### Exercise #####

# Define the policy network
class PolicyNetwork(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        super().__init__()
        assert(state_dim == 8)
        self.net = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim),
            nn.Softmax(dim=-1)
        )
        self.to(device)
    
    def forward(self, state):
        return self.net(state)

### Task 10: The Agent

Use the policy of the agent to get action probabilities. Choose an action according to the probabilities and return the action, as well as the (natural) logarithm of its probability. 

**Hint:** use `torch.distributions.Categorical`

In [88]:
##### Exercise #####

class REINFORCE:
    """REINFORCE (Monte Carlo Policy Gradient) algorithm implementation"""
    def __init__(self, state_dim, action_dim, hidden_dim=128, lr=0.001, gamma=0.99):
        self.policy = PolicyNetwork(state_dim, action_dim, hidden_dim).to(device)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr)
        self.gamma = gamma
        
    def act(self, state, greedy=False):
        """Select an action from the policy distribution
        If greedy is True, select the action with the highest probability
        """
        state = torch.FloatTensor(state).to(device).unsqueeze(0)
        
        ## YOUR CODE HERE
        # Get the action probabilities from the policy network
        action_probs = self.policy(state)
        if greedy:
            action = action_probs.argmax(dim=1).item()
            log_prob = torch.log(action_probs[0, action])
        else:
            # Sample an action from the policy distribution
            m = Categorical(action_probs)
            action = m.sample().item()
            log_prob = m.log_prob(action)
        return action, log_prob


In [89]:
##### Solution #####

class REINFORCE:
    """REINFORCE (Monte Carlo Policy Gradient) algorithm implementation"""
    def __init__(self, state_dim, action_dim, hidden_dim=128, lr=0.001, gamma=0.99):
        self.policy = PolicyNetwork(state_dim, action_dim, hidden_dim).to(device)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr)
        self.gamma = gamma
        
    def act(self, state, greedy=False):
        """Select an action from the policy distribution
        If greedy is True, select the action with the highest probability"""
        state = torch.FloatTensor(state).to(device).unsqueeze(0)
        probs = self.policy(state)
        if greedy:
            action = torch.argmax(probs, dim=-1)
            log_prob = torch.log(probs[0, action])
            return action.item(), log_prob
        else:
            m = Categorical(probs)
            action = m.sample()
            return action.item(), m.log_prob(action)

The below function runs a single episode, storing the sequence of states and actions for training

In [90]:
def run_episode(env, agent, training=True):
    """
    Run a single episode of the environment with the given agent.
    """
    state, _ = env.reset()
    rewards = []
    log_probs = []
    done = False
    
    while not done:
        action, log_prob = agent.act(state, greedy=not training)
        next_state, reward, terminated, truncated, _ = env.step(action)
        
        rewards.append(reward)
        log_probs.append(log_prob)
        
        state = next_state
        done = terminated or truncated
    
    if training:
        return sum(rewards), log_probs, rewards
    else:
        return sum(rewards)

### Task 11: Updating the Policy

1. Convert rewards to dicounted future returns:
    - Given a list of rewards $[r_1, \dots, r_T]$, compute the list of discounted future returns where $R_t = \sum_{j=t}^T \gamma^{j-t} r_j$
    - Hint: to do this efficiently, start from the end of the episode. 
    - Convert the result to a tensor, and normalize it by substracting the mean and dividing by the standard deviation. This improves training stability. 

2. Calculate policy loss
    - The loss at time $t$ is defined as $\text{loss}_t = - \log \pi_\theta(a_t \mid s_t) \cdot R_t$. 
    - The minus sign appears because we are minimizing the loss, even though the underlying objective is to maximize returns.

3. Backpropagate and update policy
    - Zero the gradients, perform backpropagation on the total policy loss, and take one optimizer step.
    - The function does not return anything. 

In [97]:
##### Exercise #####

def update_policy(agent, log_probs, rewards):
    """Update policy parameters using policy gradient"""
    
    ## YOUR CODE HERE

    # 1. Convert rewards to discounted future returns
    rewards = rewards[::-1]
    discounted_rewards = []
    R = 0
    for reward in rewards:
        R = reward + agent.gamma * R
        discounted_rewards.append(R)
    discounted_rewards = discounted_rewards[::-1]
    discounted_rewards = torch.FloatTensor(discounted_rewards).to(device)
    discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-8)
    
    # 2. Calculate policy loss
    policy_loss = torch.tensor(0.0, device=device, requires_grad=True)
    for i in range(len(log_probs)):
        policy_loss = policy_loss - log_probs[i] * discounted_rewards[i]


    # 3. Perform backpropagation and update the policy network
    agent.optimizer.zero_grad()
    policy_loss.backward()
    agent.optimizer.step()



The below function is used to train the agent:

In [98]:
def train_agent(env, agent, num_episodes=2000, print_every=100, test_every=100, num_tests=10):
    """Train the agent and periodically evaluate its performance"""
    episode_rewards = []
    test_rewards = []
    
    for episode in range(1, num_episodes+1):
        # Training episode
        total_reward, log_probs, rewards = run_episode(env, agent, training=True)
        update_policy(agent, log_probs, rewards)
        
        # Track metrics
        episode_rewards.append(total_reward)
        
        # Periodic evaluation
        if episode % test_every == 0:
            test_reward = np.mean([run_episode(env, agent, training=False) for _ in range(num_tests)])
            test_rewards.append((episode, test_reward))
            print(f"Test Reward: {test_reward:.2f}")
        
        # Periodic progress display
        if episode % print_every == 0:
            # Calculate moving average of rewards
            window = min(100, len(episode_rewards))
            avg_reward = np.mean(episode_rewards[-window:])
            
            print(f"Episode {episode}/{num_episodes}, Avg Reward: {avg_reward:.2f}")
    
    # Final evaluation
    final_reward = np.mean([run_episode(env, agent, training=False) for _ in range(num_tests)])
    print(f"\nTraining completed. Final average reward over {num_tests} episodes: {final_reward:.2f}")
    
    return episode_rewards, test_rewards#, losses

We are ready to train the agent. A good solution should achieve an average reward of 100

In [99]:
# Create and train a REINFORCE agent
agent = REINFORCE(state_dim, action_dim, hidden_dim=128, lr=0.001, gamma=0.99)
rewards, test_rewards = train_agent(env, agent, num_episodes=1500, print_every=100, test_every=200, num_tests=10)

The below code can be used to generate a gif of your agent: 

In [100]:
import imageio
import os
import shutil
from IPython.display import display, Image


def generate_gif(env, agent, n_frames=100, filename='animation', display_result=True, fps=50, image_folder='frames_temp'):
    frames = []
    os.makedirs(image_folder, exist_ok=True)

    env = NoWindowWrapper(env)
    observation = env.reset()[0]
    total_reward = 0
    for i in range(n_frames):  
        action = agent.act(observation, greedy=True)[0]
        next_observation, reward, done, _, _ = env.step(action)
        total_reward += reward
        frame = env.render()
        observation = next_observation
        frames.append(frame)
        
        # Save the frame as an image
        image_path = f"{image_folder}/frame_{i:04d}.png"
        imageio.imwrite(image_path, frame)
        
        if done:
            print(f"Total reward: {total_reward}")
            observation = env.reset()[0]
            total_reward = 0

    # Create a GIF from the saved frames
    image_files = [f"{image_folder}/frame_{i:04d}.png" for i in range(len(frames))]
    images = [imageio.v2.imread(image_file) for image_file in image_files]
    duration = 1.0 / fps
    imageio.mimsave(filename + '.gif', images, duration=duration)
    shutil.rmtree(image_folder)
    if display_result:
        display(Image(filename='animation.gif'))


generate_gif(env, agent, n_frames=1000)